In [ ]:

           #-----------------------FAKE NEWS DETECTION------------------------#
!pip install -q pyngrok flask scikit-learn joblib nltk pandas pyngrok
!ngrok config add-authtoken 2vH3CdcGgUxPDqoE8lXwA6NFBEv_4yCEdukGNc29reHrXcRP5
import pandas as pd
import re
import nltk
import joblib
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from pyngrok import ngrok
from flask import Flask, request, jsonify,render_template

from google.colab import drive
drive.mount('/content/drive')

# Load dataset
url='https://github.com/huzaifaafzal569/intership_dev/blob/main/gihub_files/Fake.csv'
df = pd.read_csv(url)
print(df.head())

# Check for missing values in 'text' column
df['Text'] = df['Text'].fillna("")  # Replace NaN values with empty strings

nltk.download('stopwords')
nltk.download('wordnet')

# Function to clean text
def clean_text(text):
    text = re.sub(r'\W', ' ', text)  # Remove special characters
    text = text.lower() # Convert all characters into lower case
    words = text.split() # split the words from whole text
    words = [word for word in words if word not in stopwords.words('english')] # a list words splitted from text
    lemmatizer = WordNetLemmatizer() # limit the words into their base like: "Running -> Run"
    words = [lemmatizer.lemmatize(word) for word in words] # list of words converted into base.
    return ' '.join(words) # after cleaning join all the words.

print("Cleaning text data...")
df['clean_text'] = df['Text'][:500].apply(clean_text) # sliced data-set.
df['clean_text'] = df['clean_text'].fillna("")  # Ensure no NaN values
print("Text data cleaned successfully!")

# Convert text into numerical form using TF-IDF
vectorizer = TfidfVectorizer(max_features=500) # (Term Frequency-Inverse Document Frequency) TD-IDF
X = vectorizer.fit_transform(df['clean_text'])
y = df['label'].map({'Fake': 0, 'Real': 1})  # Convert labels to binary values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # Split the data set into 80% training & 20% testing.

# Train model
model = MultinomialNB() # Create the model
model.fit(X_train, y_train) # Train the model using training data

# Evaluate model
y_pred = model.predict(X_test) # Make predictions on the test set
print("Accuracy:", accuracy_score(y_test, y_pred))

# Save model and vectorizer
joblib.dump(model, "model.pkl")
joblib.dump(vectorizer, "vectorizer.pkl")

# Flask API
app = Flask(__name__, template_folder='/content/drive/MyDrive/MyProjects/templates')

# Load trained model & vectorizer
model = joblib.load("model.pkl")
vectorizer = joblib.load("vectorizer.pkl")

@app.route('/')  # Home route for the form
def home():
    return render_template("index.html")  # Render a webpage

@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        data = request.form['text']  # Get input from web form
        transformed_text = vectorizer.transform([data])
        prediction = model.predict(transformed_text)[0]
        result = 'Fake' if prediction == 0 else 'Real'
        return render_template("index.html", prediction_text=f"The news is: {result}")

public_url = ngrok.connect(5000)
print(f" * ngrok tunnel running at: {public_url}")
app.run(port=5000)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
